# Data Scrapping 

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pickle
import re
from googlesearch import search
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup

In [3]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from bs4 import BeautifulSoup

software_name = [
						SoftwareName.ALERTSITE_MONITORING_BOT.value,
						SoftwareName.ANDROID.value,
						SoftwareName.AWESOMIUM.value,
						SoftwareName.BLACKBERRY.value,
						SoftwareName.CATCHPOINT_ANALYSER.value,
						SoftwareName.CHROME.value,
						SoftwareName.CHROMIUM.value,
						SoftwareName.COSMOS_CRAWLER.value,
						SoftwareName.DOTCOM_MONITOR_BOT.value,
						SoftwareName.EDGE.value,
						SoftwareName.FIREFOX.value,
						SoftwareName.FIREFOX_FOCUS.value,
						SoftwareName.GOOGLE_APP_ENGINE.value,
						SoftwareName.INTERNET_ARCHIVER_BOT.value,
						SoftwareName.INTERNET_CHANNEL.value,
						SoftwareName.INTERNET_TV.value,
						SoftwareName.K_MELEON.value,
						SoftwareName.NETSCAPE_NAVIGATOR.value,
						SoftwareName.NETSURF.value,
						SoftwareName.NOKIA.value,
						SoftwareName.OBIGO.value,
						SoftwareName.OMNIWEB.value,
						SoftwareName.ONEBROWSER.value,
						SoftwareName.OPERA.value,
						SoftwareName.OPERA_MINI.value,
						SoftwareName.PALE_MOON.value,
						SoftwareName.QUPZILLA.value,
						SoftwareName.ROCKMELT.value,
						SoftwareName.SKYFIRE.value,
						SoftwareName.SPEEDCURVE_SPEED_TESTER.value,
						SoftwareName.UC_BROWSER.value,
						SoftwareName.WEBKIT.value,
						SoftwareName.WEBTV.value,
						SoftwareName.YANDEX.value,
						SoftwareName.YANDEX_SEARCH_BOT.value,
						SoftwareName.YODAOBOT_SEARCH_BOT.value 
					]
operating_system = [
						OperatingSystem.UNIX.value,
						OperatingSystem.ANDROID.value, 
						OperatingSystem.BADA.value, 
						OperatingSystem.BEOS.value, 
						OperatingSystem.BLACKBERRY.value, 
						OperatingSystem.CHROMEOS.value, 
						OperatingSystem.FIRE_OS.value, 
						OperatingSystem.FREEBSD.value, 
						OperatingSystem.HAIKU.value, 
						OperatingSystem.IOS.value, 
						OperatingSystem.LINUX.value, 
						OperatingSystem.MAC.value, 
						OperatingSystem.MAC_OS_X.value, 
						OperatingSystem.MACOS.value, 
						OperatingSystem.OPENBSD.value, 
						OperatingSystem.SUNOS.value, 
						OperatingSystem.SYMBIAN.value, 
						OperatingSystem.WEBOS.value, 
						OperatingSystem.WINDOWS.value 
						]
user_agent_rotator = UserAgent(software_names=software_name, operating_systems=operating_system, limit=1000)
headers = {
			'User-Agent': user_agent_rotator.get_random_user_agent()
		}

## Scrapping the national health portal website to get list of diseases

In [4]:
small_alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
diseases=[]
for c in small_alpha:
    URL = 'https://www.nhp.gov.in/disease-a-z/'+c
    page = requests.get(URL,headers=headers)

    soup = BeautifulSoup(page.content, 'html.parser')
    all_diseases = soup.find('div', class_='all-disease')

    for element in all_diseases.find_all('li'):
        diseases.append(element.get_text().strip())
print(diseases)
print(len(diseases))

ConnectionError: HTTPSConnectionPool(host='www.nhp.gov.in', port=443): Max retries exceeded with url: /disease-a-z/a (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f80541751f0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
a=set(diseases)
c=list(a)
c.sort()
print(len(c))

In [ ]:
dis_symp={}
for dis in c:
  query = dis+' wikipedia'
  for sr in search(query,tld="co.in",stop=13,pause=0.5): 
    match=re.search(r'wikipedia',sr)
    filled = 0
    if match:
      wiki = requests.get(sr,headers=headers)
      print(wiki)
      soup = BeautifulSoup(wiki.content, 'html.parser')
      info_table = soup.find("table", {"class":"infobox"})
      if info_table is not None:
        for row in info_table.find_all("tr"):
          data=row.find("th",{"scope":"row"})
          if data is not None:
            data=data.get_text()
            if data=="Symptoms":
              symptom=str(row.find("td"))
              symptom = symptom.replace('.','')
              symptom = symptom.replace(';',',')
              symptom=re.sub(r'<b.*?/b>:',',',symptom) 
              symptom=re.sub(r'<a.*?>','',symptom)
              symptom=re.sub(r'</a>','',symptom)
              symptom=re.sub(r'<[^<]+?>',', ',symptom) 
              symptom=re.sub(r'\[.*\]','',symptom)
              symptom=' '.join([x for x in symptom.split() if x != ','])
              dis_symp[dis]=symptom
              filled = 1
              break
    if filled==1:
      break

In [ ]:
temp_list=[]
tmp_dict=dict()
for key,value in dis_symp.items():
  if value not in temp_list:
    tmp_dict[key]=value
    temp_list.append(value)
  else:
    print(key)

dis_symp = tmp_dict

import pandas as pd
seriesObject  = pd.Series(dis_symp,index=dis_symp.keys())
dataFrame = pd.DataFrame(seriesObject)
dataFrame
dataFrame.to_csv('file1.csv')


In [10]:
d1=pd.read_csv('file1.csv',names=['diseases','symptoms'])
d1=dis_symp.dropna()
d1

,diseases,symptoms
1,Abdominal Pain,"Diarrhea, constipation, abdominal pain,"
2,Abnormal uterine bleeding,"Irregular, abnormally frequent, prolonged, or ..."
3,Acquired Capillary Haemangioma of Eyelid,"Raised red or blue lesion,"
4,Acquired Immuno Deficiency Syndrome,"Flu-like illness,"
5,Acute encephalitis syndrome,"Headache, fever, confusion, stiff neck, vomiting,"
...,...,...
190,Xerophthalmia,"Night blindness,"
191,Yaws,"Hard swelling of the skin, ulcer, joint and bo..."
192,Yellow Fever,"Fever, chills, muscle pain, headache, yellow s..."
193,Zika virus disease,"Fever, red eyes, joint pain, headache, maculop..."


# Dataset Creation 

In [5]:
import pickle
import re
from googlesearch import search
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import time

In [10]:
# import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/shyam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/shyam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import csv
import pickle
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from itertools import combinations
from time import time
from nltk.tokenize import RegexpTokenizer
from collections import OrderedDict 

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

dis_symp=pd.read_csv('file1.csv',names=['diseases','symptoms'])
dis_symp=dis_symp.dropna()

total_symptoms = set() 
diseases_symptoms_cleaned = OrderedDict() 




In [12]:
dis_symp

,diseases,symptoms
1,Abdominal Pain,"Diarrhea, constipation, abdominal pain,"
2,Abnormal uterine bleeding,"Irregular, abnormally frequent, prolonged, or ..."
3,Acquired Capillary Haemangioma of Eyelid,"Raised red or blue lesion,"
4,Acquired Immuno Deficiency Syndrome,"Flu-like illness,"
5,Acute encephalitis syndrome,"Headache, fever, confusion, stiff neck, vomiting,"
...,...,...
190,Xerophthalmia,"Night blindness,"
191,Yaws,"Hard swelling of the skin, ulcer, joint and bo..."
192,Yellow Fever,"Fever, chills, muscle pain, headache, yellow s..."
193,Zika virus disease,"Fever, red eyes, joint pain, headache, maculop..."


In [13]:
di={}

for x in range(1,193):
    di[dis_symp['diseases'][x]]=dis_symp['symptoms'][x]

di

{'Abdominal Pain': 'Diarrhea, constipation, abdominal pain,',
 'Abnormal uterine bleeding': 'Irregular, abnormally frequent, prolonged, or excessive amounts of uterine bleeding,',
 'Acquired Capillary Haemangioma of Eyelid': 'Raised red or blue lesion,',
 'Acquired Immuno Deficiency Syndrome': 'Flu-like illness,',
 'Acute encephalitis syndrome': 'Headache, fever, confusion, stiff neck, vomiting,',
 'Adult Inclusion Conjunctivitis': 'Reddish eye, scratchiness,',
 'Alcohol Abuse and Alcoholism': 'Drinking large amounts of alcohol over a long period, difficulty cutting down, acquiring and drinking alcohol taking up a lot of time, usage resulting in problems, withdrawal occurring when stopping,',
 'Alopecia (hair loss)': 'Areas of hair loss, usually on the scalp,',
 "Alzheimer's Disease": 'Memory loss, problems with language, disorientation, mood swings,',
 'Amaurosis Fugax': 'Temporary fleeting of vision in one or both eyes,',
 'Amblyopia': 'Decreased vision,',
 'Amoebiasis': 'Bloody diar

In [14]:
for key in (di.keys()):
    value = di[key]
    list_sym = re.sub(r"\[\S+\]", "", value).lower().split(',')
    temp_sym = list_sym
    list_sym = []
    for sym in temp_sym:
        if len(sym.strip())>0:
            list_sym.append(sym.strip())
    
    if "none" in list_sym: 
        list_sym.remove("none");
    if len(list_sym)==0:
        continue
    temp = list()
    for sym in list_sym:
        sym=sym.replace('-',' ')
        sym=sym.replace("'",'')
        sym=sym.replace('(','')
        sym=sym.replace(')','')
        total_symptoms.add(sym)
        temp.append(sym)
    diseases_symptoms_cleaned[key] = temp
    
total_symptoms = list(total_symptoms)
total_symptoms.sort()
total_symptoms=['label_dis']+total_symptoms

dfn_list = []

for item in total_symptoms: 
    if item not in dfn_list: 
        dfn_list.append(item) 

df_norm = pd.DataFrame(columns=dfn_list)


for key, values in diseases_symptoms_cleaned.items():
    
    row_norm = dict({x:0 for x in total_symptoms})
    for sym in values:
        row_norm[sym] = 1
    row_norm['label_dis']=key
    #print(pd.Series(row_norm))
    df_norm = df_norm.append(pd.Series(row_norm),ignore_index=True)
    
#print(df_norm)
df_norm.to_csv("dis_dataset.csv",index=None)
